In [1]:
import math
import os
import pandas as pd
import numpy as np

import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from tqdm import tqdm_notebook, trange

import torch
import torch.nn as nn
import torch.nn.functional as F

train_path = './input/hy_round1_train_20200102'
test_path = './input/hy_round1_testA_20200102'

In [2]:
train_files = os.listdir(train_path)
test_files = os.listdir(test_path)

print(f"训练样本：{len(train_files)}")
print(f"测试样本：{len(test_files)}")

训练样本：7000
测试样本：2000


In [3]:
df = pd.read_csv(f'{train_path}/6966.csv')
print(f"作业类型：{df['type'].unique()}")
print(f"数据维度：{df.shape}")

作业类型：['围网']
数据维度：(389, 7)


In [4]:
def df_angle_handle(dataframe):
    if '方向' in dataframe.columns:
        sin_list = []
        cos_list = []
        for angle in dataframe['方向'].values:
            sin_list.append(math.sin(math.radians(angle)))
            cos_list.append(math.cos(math.radians(angle)))
        dataframe['sin_angle'] = sin_list
        dataframe['cos_angle'] = cos_list
        dataframe = dataframe.drop('方向', axis=1)
    else:
        print("没有方向列或已经处理完毕")
    return dataframe

In [5]:
df = df_angle_handle(df)

In [6]:
df = df.drop('time', axis=1)

In [7]:
# 围网
df['type'] = 1

In [8]:
df

,渔船ID,x,y,速度,type,sin_angle,cos_angle
0,6966,6.265902e+06,5.279254e+06,0.11,1,-0.809017,0.587785
1,6966,6.265902e+06,5.279254e+06,0.00,1,0.000000,1.000000
2,6966,6.265902e+06,5.279254e+06,0.00,1,0.000000,1.000000
3,6966,6.265902e+06,5.279254e+06,0.00,1,0.000000,1.000000
4,6966,6.265902e+06,5.279254e+06,0.32,1,0.766044,-0.642788
...,...,...,...,...,...,...,...
384,6966,6.312061e+06,5.245699e+06,2.10,1,0.173648,-0.984808
385,6966,6.312466e+06,5.246134e+06,0.49,1,-0.798636,-0.601815
386,6966,6.312771e+06,5.246461e+06,1.78,1,-0.999391,-0.034899
387,6966,6.312974e+06,5.246788e+06,0.49,1,-0.422618,-0.906308


In [23]:
target = torch.tensor(1)
print(target)

tensor(1)


In [17]:
train_X = torch.tensor(df.drop(['渔船ID', 'type'], axis=1).values.astype(np.float32))
train_y = torch.tensor(df['type'].values)

print(train_X)
print(train_y)
print(train_X.shape[0])

tensor([[ 6.2659e+06,  5.2793e+06,  1.1000e-01, -8.0902e-01,  5.8779e-01],
        [ 6.2659e+06,  5.2793e+06,  0.0000e+00,  0.0000e+00,  1.0000e+00],
        [ 6.2659e+06,  5.2793e+06,  0.0000e+00,  0.0000e+00,  1.0000e+00],
        ...,
        [ 6.3128e+06,  5.2465e+06,  1.7800e+00, -9.9939e-01, -3.4899e-02],
        [ 6.3130e+06,  5.2468e+06,  4.9000e-01, -4.2262e-01, -9.0631e-01],
        [ 6.3133e+06,  5.2470e+06,  5.9000e-01, -9.9863e-01, -5.2336e-02]])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

Time Series Prediction using LSTM with PyTorch in Python

https://www.kaggle.com/artgor/basic-pytorch-lstm

kaggle

In [18]:
class LSTM(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim=64, target_size=3, num_layers=2):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=1)
        self.fc = nn.Linear(hidden_dim, target_size)
    
    def forward(self, x):
        lstm_out, (hidden, cell) = self.lstm(x.view(x.shape[0], 1, -1))
        # output of shape (seq_len, batch, num_directions * hidden_size)
        # output.size() --> 100, 28, 100
        output = self.fc(lstm_out[:, -1, :])
        return output

In [19]:
model = LSTM(input_dim=5)

In [20]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.1)

In [21]:
model.train()
for epoch in trange(3):
    model.zero_grad()

    target = train_y
    tag_scorers = model(train_X)

    loss = loss_function(tag_scorers, target)
    loss.backward()
    optimizer.step()

KeyboardInterrupt: 

In [22]:
loss_fn = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)  

In [ ]:
model.train()
